## Week 3 assignment: Segmenting and Clustering Neighborhoods in Toronto
### Part 3: explore and cluster Toronto neighborhoods
#### Bostjan Keber
#### 2019/01/29

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Load the data from previus exercice into pandas dataframe

In [8]:
df = pd.read_csv('TorontoPBNLoc.csv')
df = df.drop(df.columns[0], axis=1)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811650,-79.195561
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785605,-79.158701
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175299
3,M1G,Scarborough,Woburn,43.768216,-79.217610
4,M1H,Scarborough,Cedarbrae,43.769608,-79.239440


In [9]:
#Check the content of the Toronto dataframe
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Toronto


In [10]:
address = 'Toronto'

#geolocator = Nominatim(user_agent="ny_explorer")
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [12]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [17]:
#let's explore Scarborough borough of Toronto
Scarborough_data = df[df['Borough'] == 'Scarborough'].reset_index(drop=True)
Scarborough_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811650,-79.195561
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785605,-79.158701
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175299
3,M1G,Scarborough,Woburn,43.768216,-79.217610
4,M1H,Scarborough,Cedarbrae,43.769608,-79.239440


In [18]:
#get geo coordinates of Scarborough, Toronto
address = 'Scarborough, Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough, Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough, Toronto are 43.773077, -79.257774.


#### Let's visualize Scarborough, Toronto

In [19]:
# create map of Manhattan using latitude and longitude values
map_Scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Scarborough_data['Latitude'], Scarborough_data['Longitude'], Scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Scarborough)  
    
map_Scarborough

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define FourSquare credentials and version

In [20]:
CLIENT_ID = 'TZCKCZNMYA5VRVPEGM15CRQ1JQP4JLAE2EYAVH1IX5Z5OY5H' # your Foursquare ID
CLIENT_SECRET = 'LXSH0MF3AXP4WK5H4FBAX2E0O4TAE2CDII3XQVKI2IZ4CVMX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TZCKCZNMYA5VRVPEGM15CRQ1JQP4JLAE2EYAVH1IX5Z5OY5H
CLIENT_SECRET:LXSH0MF3AXP4WK5H4FBAX2E0O4TAE2CDII3XQVKI2IZ4CVMX


#### Let's explore the first neighborhood in our dataframe.

In [21]:
Scarborough_data.loc[0, 'Neighborhood']

'Rouge, Malvern'

Get the neighborhood's latitude and longitude values.

In [22]:
neighborhood_latitude = Scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Scarborough_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge, Malvern are 43.811650000000036, -79.19556138899999.


#### Now, let's get the top 100 venues that are in Rouge, Malvern within a radius of 500 meters.
First, let's create the GET request URL. Name your URL url.

In [23]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=TZCKCZNMYA5VRVPEGM15CRQ1JQP4JLAE2EYAVH1IX5Z5OY5H&client_secret=LXSH0MF3AXP4WK5H4FBAX2E0O4TAE2CDII3XQVKI2IZ4CVMX&v=20180605&ll=43.811650000000036,-79.19556138899999&radius=500&limit=100'

In [24]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c50dec0f594df20f2a245f6'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.81615000450004,
    'lng': -79.18933705248097},
   'sw': {'lat': 43.80714999550003, 'lng': -79.20178572551902}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5c0af1c44a1cc0002cc5537b',
       'name': 'Scarborough Bad Credit Car Loans',
       'location': {'address': '10 Thornmont Drive',
        'lat': 43.80812307,
        'lng': -79.19913259,
        'labeledLatLngs': [{'lab

In [25]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [26]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Scarborough Bad Credit Car Loans,Rental Car Location,43.808123,-79.199133


In [27]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


### 2. Explore Neighborhoods in Scarborough, Toronto

Let's create a function to repeat the same process to all the neighborhoods of Scarborough

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called Scarborough_venues.

In [29]:
Scarborough_venues = getNearbyVenues(names=Scarborough_data['Neighborhood'],
                                   latitudes=Scarborough_data['Latitude'],
                                   longitudes=Scarborough_data['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge


#### Let's check the size of the resulting dataframe

In [30]:
print(Scarborough_venues.shape)
Scarborough_venues.head()

(97, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.811650,-79.195561,Scarborough Bad Credit Car Loans,43.808123,-79.199133,Rental Car Location
1,"Highland Creek, Rouge Hill, Port Union",43.785605,-79.158701,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.765690,-79.175299,Homestead Roofing Repair,43.765140,-79.178663,Construction & Landscaping
3,"Guildwood, Morningside, West Hill",43.765690,-79.175299,Heron Park Community Centre,43.768867,-79.176958,Gym / Fitness Center
4,"Guildwood, Morningside, West Hill",43.765690,-79.175299,Heron Park,43.769327,-79.177201,Park


Let's check how many venues were returned for each neighborhood

In [31]:
Scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,15,15,15,15,15,15
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Birch Cliff, Cliffside West",7,7,7,7,7,7
Cedarbrae,2,2,2,2,2,2
"Clairlea, Golden Mile, Oakridge",9,9,9,9,9,9
"Clarks Corners, Sullivan, Tam O'Shanter",12,12,12,12,12,12
"Cliffcrest, Cliffside, Scarborough Village West",11,11,11,11,11,11
"Dorset Park, Scarborough Town Centre, Wexford Heights",3,3,3,3,3,3
"East Birchmount Park, Ionview, Kennedy Park",6,6,6,6,6,6


#### Let's find out how many unique categories can be curated from all the returned venues

In [32]:
print('There are {} uniques categories.'.format(len(Scarborough_venues['Venue Category'].unique())))

There are 54 uniques categories.


### 3. Analyze Each Neighborhood

In [33]:
# one hot encoding
Scarborough_onehot = pd.get_dummies(Scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Scarborough_onehot['Neighborhood'] = Scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Scarborough_onehot.columns[-1]] + list(Scarborough_onehot.columns[:-1])
Scarborough_onehot = Scarborough_onehot[fixed_columns]

Scarborough_onehot.head()

,Neighborhood,Auto Garage,Bakery,Bar,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,General Entertainment,Gift Shop,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hobby Shop,Indian Restaurant,Intersection,Japanese Restaurant,Korean Restaurant,Liquor Store,Other Great Outdoors,Park,Pharmacy,Pizza Place,Playground,Pool,Rental Car Location,Restaurant,Sandwich Place,Shanghai Restaurant,Shipping Store,Shopping Mall,Skating Rink,Soccer Field,Supermarket,Sushi Restaurant,Thai Restaurant,Thrift / Vintage Store,Trail,Train Station,Vietnamese Restaurant,Wings Joint
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [34]:
Scarborough_onehot.shape

(97, 55)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [35]:
Scarborough_grouped = Scarborough_onehot.groupby('Neighborhood').mean().reset_index()
Scarborough_grouped

,Neighborhood,Auto Garage,Bakery,Bar,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,General Entertainment,Gift Shop,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hobby Shop,Indian Restaurant,Intersection,Japanese Restaurant,Korean Restaurant,Liquor Store,Other Great Outdoors,Park,Pharmacy,Pizza Place,Playground,Pool,Rental Car Location,Restaurant,Sandwich Place,Shanghai Restaurant,Shipping Store,Shopping Mall,Skating Rink,Soccer Field,Supermarket,Sushi Restaurant,Thai Restaurant,Thrift / Vintage Store,Trail,Train Station,Vietnamese Restaurant,Wings Joint
0,Agincourt,0.000000,0.066667,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.133333,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.066667,0.000000,0.000000,0.0,0.066667,0.0,0.00,0.000000,0.066667,0.000000,0.133333,0.066667,0.000000,0.133333,0.066667,0.000000,0.000000,0.0,0.00,0.066667,0.000000
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.666667,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.0,0.00,0.000000,0.000000
2,"Birch Cliff, Cliffside West",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.142857,0.000000,0.142857,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.142857,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000
3,Cedarbrae,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.00,0.000000,0.000000
4,"Clairlea, Golden Mile, Oakridge",0.000000,0.222222,0.0,0.000000,0.222222,0.111111,0.000000,0.000000,0.000000,0.222222,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.111111,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000
5,"Clarks Corners, Sullivan, Tam O'Shanter",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.083333,0.000000,0.083333,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.083333,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.166667,0.166667,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.0,0.00,0.000000,0.000000
6,"Cliffcrest, Cliffside, Scarborough Village West",0.000000,0.000000,0.0,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.181818,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0

Let's confirm the new size

In [37]:
Scarborough_grouped.shape

(16, 55)

#### Let's print each neighborhood along with the top 5 most common venues

In [38]:
num_top_venues = 5

for hood in Scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Scarborough_grouped[Scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                 venue  freq
0          Supermarket  0.13
1        Shopping Mall  0.13
2   Chinese Restaurant  0.13
3     Sushi Restaurant  0.07
4  Shanghai Restaurant  0.07


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                 venue  freq
0             Pharmacy  0.67
1     Sushi Restaurant  0.33
2          Auto Garage  0.00
3       Sandwich Place  0.00
4  Japanese Restaurant  0.00


----Birch Cliff, Cliffside West----
              venue  freq
0              Park  0.14
1          Gym Pool  0.14
2               Gym  0.14
3  Business Service  0.14
4   College Stadium  0.14


----Cedarbrae----
                 venue  freq
0                Trail   0.5
1           Playground   0.5
2          Auto Garage   0.0
3       Sandwich Place   0.0
4  Japanese Restaurant   0.0


----Clairlea, Golden Mile, Oakridge----
          venue  freq
0        Bakery  0.22
1      Bus Line  0.22
2   Coffee Shop  0.22
3  Intersection  0.11
4   Bus Station  0.11


---

#### Let's put that into a pandas dataframe

In [39]:
# First, let's write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
#Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Scarborough_grouped['Neighborhood']

for ind in np.arange(Scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Shopping Mall,Chinese Restaurant,Supermarket,Bakery,Discount Store,Vietnamese Restaurant,Sushi Restaurant,Pool,Skating Rink,Park
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Pharmacy,Sushi Restaurant,Wings Joint,Department Store,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store
2,"Birch Cliff, Cliffside West",Gym,General Entertainment,Business Service,Skating Rink,Park,College Stadium,Gym Pool,Wings Joint,Discount Store,Grocery Store
3,Cedarbrae,Trail,Playground,Wings Joint,Department Store,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store
4,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Coffee Shop,Intersection,Bus Station,Soccer Field,Wings Joint,Electronics Store,Gym,Grocery Store


### 4. Cluster Neighborhoods
Run k-means to cluster the neighborhood into 3 clusters.

In [88]:
# set number of clusters
kclusters = 5

Scarborough_grouped_clustering = Scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:16]

array([0, 2, 0, 4, 0, 0, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [93]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Scarborough_merged = Scarborough_data

Scarborough_merged = Scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Scarborough_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.811650,-79.195561,3.0,Rental Car Location,Wings Joint,Department Store,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store,Fried Chicken Joint
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785605,-79.158701,1.0,Bar,Wings Joint,Department Store,Gym / Fitness Center,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175299,0.0,Gym / Fitness Center,Park,Construction & Landscaping,Department Store,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store
3,M1G,Scarborough,Woburn,43.768216,-79.217610,0.0,Korean Restaurant,Business Service,Park,Coffee Shop,Wings Joint,Discount Store,Gym,Grocery Store,Golf Course,Gift Shop
4,M1H,Scarborough,Cedarbrae,43.769608,-79.239440,4.0,Trail,Playground,Wings Joint,Department Store,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store


#### Visualize clusters

In [81]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Scarborough_merged['Latitude'], Scarborough_merged['Longitude'], Scarborough_merged['Neighborhood'], Scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 5. Examine Clusters
Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [82]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 0, Scarborough_merged.columns[[1] + list(range(5, Scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,0.0,Gym / Fitness Center,Park,Construction & Landscaping,Department Store,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store
3,Scarborough,0.0,Korean Restaurant,Business Service,Park,Coffee Shop,Wings Joint,Discount Store,Gym,Grocery Store,Golf Course,Gift Shop
5,Scarborough,0.0,Restaurant,Train Station,Indian Restaurant,Grocery Store,Convenience Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store,Fried Chicken Joint
6,Scarborough,0.0,Discount Store,Convenience Store,Chinese Restaurant,Coffee Shop,Department Store,Gym / Fitness Center,Gym,Grocery Store,Golf Course,Gift Shop
7,Scarborough,0.0,Bakery,Bus Line,Coffee Shop,Intersection,Bus Station,Soccer Field,Wings Joint,Electronics Store,Gym,Grocery Store
8,Scarborough,0.0,Fast Food Restaurant,Wings Joint,Discount Store,Burger Joint,Furniture / Home Store,Liquor Store,Coffee Shop,Pharmacy,Sandwich Place,Pizza Place
9,Scarborough,0.0,Gym,General Entertainment,Business Service,Skating Rink,Park,College Stadium,Gym Pool,Wings Joint,Discount Store,Grocery Store
10,Scarborough,0.0,Bakery,Gift Shop,Shipping Store,Wings Joint,Department Store,Gym,Grocery Store,Golf Course,General Entertainment,Furniture / Home Store
11,Scarborough,0.0,Auto Garage,Intersection,Convenience Store,Bar,Burger Joint,Bakery,Gym,Grocery Store,Golf Course,Gift Shop
12,Scarborough,0.0,Shopping Mall,Chinese Restaurant,Supermarket,Bakery,Discount Store,Vietnamese Restaurant,Sushi Restaurant,Pool,Skating Rink,Park


#### Cluster 2

In [83]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 1, Scarborough_merged.columns[[1] + list(range(5, Scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,1.0,Bar,Wings Joint,Department Store,Gym / Fitness Center,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store


#### Cluster 3

In [84]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 2, Scarborough_merged.columns[[1] + list(range(5, Scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,2.0,Pharmacy,Sushi Restaurant,Wings Joint,Department Store,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store


#### Cluster 4

In [85]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 3, Scarborough_merged.columns[[1] + list(range(5, Scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,3.0,Rental Car Location,Wings Joint,Department Store,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store,Fried Chicken Joint


#### Cluster 5

In [86]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 4, Scarborough_merged.columns[[1] + list(range(5, Scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Scarborough,4.0,Trail,Playground,Wings Joint,Department Store,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store
